In [ ]:
from qiskit import QuantumCircuit, transpile, Aer, execute
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from qiskit.circuit.library import GroverOperator
from qiskit.algorithms import Shor
from qiskit.utils import QuantumInstance
from qiskit.circuit import QuantumRegister, ClassicalRegister
import random

In [ ]:
# Генерация случайного ключа
def generate_key(length):
    return [random.choice([0, 1]) for _ in range(length)]

# Случайные базы для Алисы и Боба
alice_bases = generate_key(10)
bob_bases = generate_key(10)

# Генерация квантовой цепи
key_bits = generate_key(10)
qc_list = []
for i in range(10):
    qc = QuantumCircuit(1, 1)
    if alice_bases[i] == 1:
        qc.h(0)
    if key_bits[i] == 1:
        qc.x(0)
    if bob_bases[i] == 1:
        qc.h(0)
    qc.measure(0, 0)
    qc_list.append(qc)

# Результаты передаются для последующей проверки совпадений базисов
# Здесь можно включить этап симуляции с передачей и сравнением
